In [ ]:
("C:/Users/Yusuf/Documents/Delete/dens_df.csv",index=0)
dest_raw_df.to_csv("C:/Users/Yusuf/Documents/Delete/dest_raw_df.csv",index=0)
dens_df.to_csv("C:/Users/Yusuf/Documents/Delete/dens_df.csv",index=0)
path = "C:\\Users\\Yusuf\\Desktop\\GA-Tech\\CSE6242\\Project\\KotoMaterials\\KotoMaterials\\"
data = [json.loads(line) for line in open(path+'dwellTimes2.json', 'r')]
#Convert list of json's into data frame
df = pd.DataFrame(columns = ['seq', 'start_timestamp', 'end_timestamp','vin'])
column_names_dict ={'index':'seq',0:'start_timestamp',1:"end_timestamp"}
for i in range(len(data)):
     temp_df = pd.DataFrame(data[i]['dwell']).transpose().reset_index().rename(columns=column_names_dict)
     temp_df['vin'] = data[i]["_id"]
     df = df.append(temp_df, ignore_index=True)
     #Clean data types
df = df.astype({'seq':'int64'})
df['start_timestamp'] = pd.to_datetime(df['start_timestamp'])
df['end_timestamp'] = pd.to_datetime(df['end_timestamp'])
#Extract additional features
df['arrival_hour'] = df["start_timestamp"].dt.hour
df['arrival_date'] = df["start_timestamp"].dt.date
df['arrival_day'] = df["start_timestamp"].dt.day_name()
df['trip_time'] = (df['end_timestamp'] - df['start_timestamp']).astype('timedelta64[m]')
#Sort and shift data to get additional features
df = df.sort_values(by =['vin','seq'])
df["next_trip_start_timestamp"] = df["start_timestamp"].shift(periods=-1)
df["next_trip_seq"] = df['seq'].shift(periods=-1)
df["dwell_time"] = (df['next_trip_start_timestamp'] - df['end_timestamp']).astype('timedelta64[m]')
df['seq_diff'] = df['next_trip_seq'] - df['seq']
path = "C:\\Users\\Yusuf\\Desktop\\GA-Tech\\CSE6242\\Project\\KotoMaterials\\KotoMaterials\\"
data_dest = [json.loads(line) for line in open(path+'destination.json', 'r')]
df_dest = pd.DataFrame(columns = ['seq', 'Lat', 'Lon','vin'])
column_names_dict ={'index':'seq',0:'Lat',1:"Lon"}
for i in range(len(data_dest)):
     temp_df = pd.DataFrame(data_dest[i]['destination']).transpose().reset_index().rename(columns=column_names_dict)
     temp_df['vin'] = data[i]["_id"]
     df_dest = df_dest.append(temp_df, ignore_index=True)
df_dest['seq'] = df_dest['seq'].astype(int)
df_dest.to_csv("C:/Users/Yusuf/Documents/Delete/dest_raw_all_df.csv")
raw_dest_dwell_df = df_dest.merge(df.reset_index(drop=True),on=['vin','seq'], how='inner')
raw_dest_dwell_viz_df = raw_dest_dwell_df[['latitude','longitude','vin','start_timestamp','end_timestamp','arrival_hour']]
raw_dest_dwell_viz_df.to_csv("C:/Users/Yusuf/Documents/Delete/raw_dest_dwell_viz_df.csv",index=0)

In [ ]:
##Potentially transfer to different sub-module #Priority 3
def single_vin_viz(vin:str='070aac90eee9f2eaf444a1ccbb894df0'):

    ##Write Doc String

    #Connect to locationCollector collection
    myclient = pymongo.MongoClient("mongodb://localhost:27017")
    mydb = myclient['kotomatic_db']
    mycol = mydb["locationCollector"]

    #Create list of document ids, which are encrypted vins
    id_list = mycol.find().distinct("_id")


    if vin == 'random':
        random_index = random.randint(0,len(id_list))
        vin = id_list[random_index]
        print(vin)

    #Find Vin locationCollector document
    filt = {'_id': vin}
    found = mycol.find_one(filt)

    #Create dataframe from document
    viz_df = pd.DataFrame(found['location']).transpose().reset_index().rename(columns = {'index':'DateTime',0:'Lat',1:'Lon'})

    #Get mapbox_token
    with open("mapbox_token.txt", 'r') as f:
        mb_token = f.read()

    #Create scatter
    fig = px.scatter_mapbox(viz_df, \
        lat = "Lat", \
        lon = "Lon", \
        color_discrete_sequence=["fuchsia"], \
        hover_name="DateTime")

    fig.update_layout(mapbox_style='dark',mapbox_accesstoken = mb_token)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()

In [ ]:
    @staticmethod
    def unique_bluetooth_devices(response_object=[], agg_bluetooth_dict={}):
        ##Write doc string
        assert len(response_object) > 0, "Empty list passed"
        #Necessary fields
        neccesary_fields = ['vin', 'phone_bluetoothconnection_deviceaddress']
        #Check if necessary fields are available
        valid_fields = all(field in response_object[0].keys() for field in neccesary_fields)
        assert valid_fields, "Not all required fields are present in the response object"
        ##Sort out scoping issue with agg_dict

        for entry in response_object:
            if entry['phone_bluetoothconnection_deviceaddress'] is not None:
                if entry['vin'] not in agg_bluetooth_dict.keys():
                    agg_bluetooth_dict[entry['vin']] = [entry['phone_bluetoothconnection_deviceaddress']]

                elif entry['phone_bluetoothconnection_deviceaddress'] not in agg_bluetooth_dict[entry['vin']]:
                    agg_bluetooth_dict[entry['vin']].append(entry['phone_bluetoothconnection_deviceaddress'])
                
                else:
                    pass
            else:
                continue

        return agg_bluetooth_dict

    @staticmethod
    def location_collector(response_object=[], agg_loc_dict={}):
        
        ##Write doc string
        assert len(response_object) > 0, "Empty list passed"
        ##Sort out scoping issue with agg_dict
        
        for entry in response_object:

            #Is even minute? #Data reduction
            time_ymd_hm = datetime.datetime.strptime(entry['timestamp'][:-4], "%Y-%m-%d %H:%M:%S")
            ts_key = time_ymd_hm.strftime("%Y-%m-%d %H:%M")
            if time_ymd_hm.minute % 3 == 0 and entry['navigation_location_coordinate_longitude'] is not None:
                if entry['vin'] not in agg_loc_dict.keys():
                    vin_key = entry['vin']
                    loc_value = [entry['navigation_location_coordinate_latitude'],entry['navigation_location_coordinate_longitude']]
                    agg_loc_dict[vin_key] = {ts_key: loc_value }
                elif ts_key not in agg_loc_dict[entry['vin']].keys():
                    loc_value = [entry['navigation_location_coordinate_latitude'],entry['navigation_location_coordinate_longitude']]
                    agg_loc_dict[entry['vin']][ts_key] = loc_value
                else:
                    continue
        return agg_loc_dict

    @staticmethod
    def location_collector_v2(response_object=[]):
        
        ##Write doc string
        #assert len(response_object) > 0, "Empty list passed"

        if len(response_object)>0:
            
            agg_loc_dict = {}
            for entry in response_object:

                #Data reduction
                time_ymd_hm = datetime.datetime.strptime(entry['timestamp'][:-4], "%Y-%m-%d %H:%M:%S")
                ts_key = time_ymd_hm.strftime("%Y-%m-%d %H:%M")
                if time_ymd_hm.minute % 3 == 0 and entry['navigation_location_coordinate_longitude'] is not None:
                    if entry['vin'] not in agg_loc_dict.keys():
                        vin_key = entry['vin']
                        loc_value = [entry['navigation_location_coordinate_latitude'],entry['navigation_location_coordinate_longitude']]
                        agg_loc_dict[vin_key] = {ts_key: loc_value }
                    elif ts_key not in agg_loc_dict[entry['vin']].keys():
                        loc_value = [entry['navigation_location_coordinate_latitude'],entry['navigation_location_coordinate_longitude']]
                        agg_loc_dict[entry['vin']][ts_key] = loc_value
                    else:
                        continue

            #Data Handler
            lc_mongo_handler(data=agg_loc_dict)
        else:
            agg_loc_dict = {}
            
        return agg_loc_dict

    @staticmethod
    def unique_bluetooth_devices_v2(response_object=[], data_handler:list=[]):
        """
        Objective:

            Take a response object generated by a kotoTelematics request function
            Iteratively generate a dictionary where VIN numbers are keys and an list of
            unique bluetooth device IDs are values. User data handler to store externally
        
        Parameters:
            
            :response_object(list):[]
                List of returned values from a koto request function
            :data_handler(list):[ubd_mongo_handler]
                Function passed to manage data storage and aggregation
        
        Return:

            :rtype: dictionary
            :rvalue: 
                Dictionary with keys equal to VIN numbers and values as a list of unique devices that
                were observed to be connected at least once


        """

        
        if len(response_object) > 0:

            #Necessary fields
            neccesary_fields = ['vin', 'phone_bluetoothconnection_deviceaddress']

            #Check if necessary fields are available
            valid_fields = all(field in response_object[0].keys() for field in neccesary_fields)
            assert valid_fields, "Not all required fields are present in the response object"

            ##Sort out scoping issue with agg_dict
            agg_bluetooth_dict={}
            for entry in response_object:
                if entry['phone_bluetoothconnection_deviceaddress'] is not None:
                    if entry['vin'] not in agg_bluetooth_dict.keys():
                        agg_bluetooth_dict[entry['vin']] = [entry['phone_bluetoothconnection_deviceaddress']]

                    elif entry['phone_bluetoothconnection_deviceaddress'] not in agg_bluetooth_dict[entry['vin']]:
                        agg_bluetooth_dict[entry['vin']].append(entry['phone_bluetoothconnection_deviceaddress'])
                    
                    else:
                        pass
                else:
                    continue

            #Data Handler
            if len(data_handler)>0:
                data_handler(data= agg_bluetooth_dict)
            else:
                pass
        else:
            agg_bluetooth_dict={}

        return agg_bluetooth_dict

    ##Create function to collect vin as document id and destinations from navi as values in array
    @staticmethod
    def destination_collector(response_object=[]):

        ##Write doc string #Priority 2

        if len(response_object) > 0:

            #Necessary fields
            neccesary_fields = ['vin', 'navigation_location_coordinate_longitude', 'navigation_location_coordinate_latitude', 'sequence']

            #Check if necessary fields are available
            valid_fields = all(field in response_object[0].keys() for field in neccesary_fields)
            assert valid_fields, "Not all required fields are present in the response object"

            agg_destination_dict = {}
            for entry in response_object:
                seq = str(entry['sequence'])
                lat = entry["navigation_location_coordinate_latitude"]
                lon = entry["navigation_location_coordinate_longitude"]
                location = [lat, lon]

                if entry['navigation_location_coordinate_longitude'] is None:
                    continue
                elif entry['vin'] in agg_destination_dict.keys():
                    agg_destination_dict[entry['vin']][seq] = location
                elif entry['vin'] not in agg_destination_dict.keys():
                    payload = {seq : location}
                    agg_destination_dict[entry['vin']] = payload

            #Data Handler
            dc_mongo_handler(data= agg_destination_dict)

        else:
            agg_destination_dict = {}

        return agg_destination_dict

 ##Create function to collect vin as document id and destinations from navi as values in array
    @staticmethod
    def destination_collector_demo(response_object=[]):

        ##Write doc string #Priority 2

        if len(response_object) > 0:

            #Necessary fields
            neccesary_fields = ['vin', 'navigation_location_coordinate_longitude', 'navigation_location_coordinate_latitude', 'sequence']

            #Check if necessary fields are available
            valid_fields = all(field in response_object[0].keys() for field in neccesary_fields)
            assert valid_fields, "Not all required fields are present in the response object"

            agg_destination_dict = {}
            for entry in response_object:
                seq = str(entry['sequence'])
                lat = entry["navigation_location_coordinate_latitude"]
                lon = entry["navigation_location_coordinate_longitude"]
                location = [lat, lon]

                if entry['navigation_location_coordinate_longitude'] is None:
                    continue
                elif entry['vin'] in agg_destination_dict.keys():
                    agg_destination_dict[entry['vin']][seq] = location
                elif entry['vin'] not in agg_destination_dict.keys():
                    payload = {seq : location}
                    agg_destination_dict[entry['vin']] = payload

                print(agg_destination_dict)

        else:
            agg_destination_dict = {}

        return agg_destination_dict